In [0]:
import theano
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

#importing the dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Read the dataset**

In this dataset, we have to consider which of the factors may play a role in someone exiting a bank. To do that we must look at all the column and infer whether it will matter in classifying a new customer or not. The information about a customer is entailed in columns 0 through 12 (RowNumber-EstimatedSalary), while the output (whether the customer exited or not) is stored in the 13th row (Exited).

For as much as we care, neither the customer ID, nor the surname should matter in classification. Therefore, we will use columns 3 (CreditScore) inclusive through the 13th column (exclusive).

Given a dataset consisiting of Bank Customer information, we are asked to build a classifier which will tell us if a customer will exit the bank or not

In [0]:
# Read the dataset
dataset = pd.read_csv('bank.csv')
dataset


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


**Drop the columns which are unique for all users like IDs**

In [0]:
#2.Drop the columns which are unique for all users like IDs (5 points)

X = dataset.iloc[:,3:13].values # Credit Score through Estimated Salary
y = dataset.iloc[:, 13].values # Exited

**Distinguish the feature and target set (5 points)**

In [0]:
# Encoding categorical (string based) data. Country: there are 3 options: France, Spain and Germany
# This will convert those strings into scalar values for analysis
print(X[:8,1], '... will now become: ')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_X_country_encoder = LabelEncoder()


#Distinguish the feature and target set (5 points)
X[:,1] = label_X_country_encoder.fit_transform(X[:,1])
print(X[:8,1])

['France' 'Spain' 'France' 'France' 'Spain' 'Spain' 'France' 'Germany'] ... will now become: 
[0 2 0 0 2 2 0 1]


In [0]:
# We will do the same thing for gender. this will be binary in this dataset
print(X[:6,2], '... will now become: ')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_X_gender_encoder = LabelEncoder()
X[:,2] = label_X_gender_encoder.fit_transform(X[:,2])
print(X[:6,2])

['Female' 'Female' 'Female' 'Female' 'Female' 'Male'] ... will now become: 
[0 0 0 0 0 1]


The Problem here is that we are treating the countries as one variable with ordinal values (0 < 1 < 2). Therefore, one way to get rid of that problem is to split the countries into respective dimensions.

In [0]:
# Converting the string features into their own dimensions. Gender doesn't matter here because its binary
countryhotencoder = OneHotEncoder(categorical_features = [1]) # 1 is the country column
X = countryhotencoder.fit_transform(X).toarray()
X

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[1.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [1.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [1.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

Gender doesn't need to go through a similar process becasue it is binary

You can now see that the first three columns represent the three countries that constituted the "country" category. We can now observe that we essentially only need two columns: a 0 on two countries means that the country has to be the one variable which wasn't included. This will save us from the problem of using too many dimensions




In [0]:
X = X[:,1:] # Got rid of Spain as a dimension. It is still there through out inferences

**Divide the data set into Train and test sets**

In [0]:
# Splitting the dataset into the Training and Testing set.
#4. Divide the data set into Train and test sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

**Normalize the train and test data (5 points)**

Feature scaling is a method used to standardize the range of independent variables or features of data. It is basically scaling all the dimensions to be even so that one independent variable does not dominate another. For example, bank account balance ranges from millions to 0, whereas gender is either 0 or 1. If one of the features has a broad range of values, the distance will be governed by this particular feature. Therefore, the range of all features should be normalized so that each feature contributes approximately proportionately to the final distance.

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
X_train

array([[-0.5698444 , -1.01460667,  1.74309049, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -1.01460667, -0.57369368, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 ,  0.98560362, -0.57369368, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [-0.5698444 ,  0.98560362, -0.57369368, ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 , -1.01460667,  1.74309049, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -1.01460667, -0.57369368, ...,  0.64259497,
        -1.03227043,  0.32472465]])

In [0]:
import keras # Test out Theano when time permits as well

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense

**Initialize & build the model**

The breakdown of the inputs for the first layer is as follows:
activiation: relu becasue we are in an input layer. uses the ReLu activation function for  ϕ 
input_dim: 11 because we span 11 dimensions in our input layer. This is needed for the first added layer. The subsequent layers's input dimensions can be inferred using the previously added layer's output dimension. The next hidden layer will know what to expect.

units: 6 nodes (number of nodes in hidden layer). Can think of this as number of nodes are in the next layer.

kernel_initializer: uniform the distribution with which we randomly initialize weights for the nodes in this layer.

In [0]:
# Initializing the ANN
classifier = Sequential()

W0901 01:48:48.582105 140188112840576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



We are going to add another layer to this model because we want to implement Deep Learning, which is an artificial Neural network with many layers.
We will make our second hidden layer also have 6 nodes, just playing with the same arithmetic we used to determine the dimensions of the first hidden layer (average of your input and output layers)  (11+1)÷2=6 .

In [0]:
# Adding the second hidden layer
# Notice that we do not need to specify input dim. 
classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform'))

Adding the output layer
The breakdown of the inputs for the output layer is as follows:
activiation: sigmoid becasue we are in an output layer. uses the Sigmoid activation function for  ϕ . This is used instead of the ReLu function becasue it generates probabilities for the outcome. We want the probability that each customer leaves the bank.

input_dim: 11 because we span 11 dimensions in our input layer. This is needed for the first added layer. The subsequent layers's input dimensions can be inferred using the previously added layer's output dimension. The next hidden layer will know what to expect.

units: 6 nodes (number of nodes in hidden layer). Can think of this as number of nodes are in the next layer.

kernel_initializer: uniform the distribution with which we randomly initialize weights for the nodes in this layer.

In [0]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 

The breakdown of the inputs for compiling is as follows:
optimizer: adam The algorithm we want to use to find the optimal set of weights in the neural networks. Adam is a very efficeint variation of Stochastic Gradient Descent.

loss: binary_crossentropy This is the loss function used within adam. This should be the logarthmic loss. If our dependent (output variable) is Binary, it is binary_crossentropy. If Categorical, then it is called categorical_crossentropy

metrics: [accuracy] The accuracy metrics which will be evaluated(minimized) by the model. Used as accuracy criteria to imporve model performance.

kernel_initializer: uniform the distribution with which we randomly initialize weights for the nodes in this layer.



In [0]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

W0901 01:51:16.315730 140188112840576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



**Print the Accuracy score and confusion matrix (5 points)**

Fitting the Neural Network
This is where we will be fitting the ANN to our training set.

The breakdown of the inputs for compiling is as follows:
X_train The independent variable portion of the data which needs to be fitted with the model.

Y_train The output portion of the data which the model needs to produce after fitting.

batch_size: How often we want to back-propogate the error values so that individual node weights can be adjusted.

nb_epochs: The number of times we want to run the entire test data over again to tune the weights. This is like the fuel of the algorithm.

In [0]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

W0901 01:51:27.004364 140188112840576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0901 01:51:27.008845 140188112840576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0901 01:51:27.082607 140188112840576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3657: The name tf.log is deprecated. Please use tf.math.log instead.

W0901 01:51:27.090188 140188112840576 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 

Epoch 1/100
8000/8000 [==============================] - 2s 201us/step - loss: 0.5734 - acc: 0.7835
Epoch 2/100
8000/8000 [==============================] - 1s 109us/step - loss: 0.4858 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 117us/step - loss: 0.4620 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.4500 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.4435 - acc: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 112us/step - loss: 0.4394 - acc: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 112us/step - loss: 0.4362 - acc: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4327 - acc: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 115us/step - loss: 0.4292 - acc: 0.7960
Epoch 10/100
8000/8000 [==============================] - 1s 109us/step - loss: 0.4262 - acc: 0.7960

Predicting the Test set results
This shows the probability of a customer leaving given the testing data. Each row in X_test corresponds to a row in Y_test

The output network should converge to an accuracy of 83.37%

In [0]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.19814971]
 [0.33269587]
 [0.1335682 ]
 ...
 [0.08980665]
 [0.15054995]
 [0.1054557 ]]


To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.

In [0]:
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


Making the Confusion Matrix

In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1521   74]
 [ 246  159]]


In [0]:
print (((cm[0][0]+cm[1][1])*100)/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0]), '% of testing data was classified correctly')

84.0 % of testing data was classified correctly
